In [ ]:
from tools.Models import TwoLink
from math import pi
import spatialgeometry as sg

robot = TwoLink()
# robot.q = [pi/4, 0]
fr = robot.fkine(robot.q)
print(fr)
J = robot.jacob0(robot.q)
print(J)

axes = sg.Axes(0.5)
print(axes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def f(t):
    return np.exp(-t) * np.cos(2*np.pi*t)


t1 = np.arange(0.0, 5.0, 0.1)
t2 = np.arange(0.0, 5.0, 0.02)


fig,axs = plt.subplots(2,1)
axs[0].plot(t1, f(t1), color='tab:blue', marker='o')
axs[0].plot(t2, f(t2), color='black')
axs[1].plot(t2, np.cos(2*np.pi*t2), color='tab:orange', linestyle='--')

In [ ]:
from tools.Models import TwoLink
robot = TwoLink()
currentPose = robot.fkine(robot.q)
print(currentPose)
currentPose.t[0:2]

In [ ]:
import numpy as np
t = []
t.append([0,1])
t.append([2,3])
print(t)
t = np.array(t)
t[:,0]

In [ ]:

import numpy as np
l = [[2,3],[3,4],[4,5]]

np.array(l)

In [ ]:
import numpy as np
from tools.Models import TwoLink, UncertantTwoLink
TwoLink()

In [ ]:
from tools.Control import Control
from roboticstoolbox.backends.PyPlot import PyPlot
from math import pi
from roboticstoolbox.tools.trajectory import *
from tools.Utils import *
from control.trajectory_control import *

brobot = UncertantTwoLink()
robot = TwoLink()
env = PyPlot()
goal = [pi/8,0]

T = 2
traj_fun = [quintic_func(robot.q[i], goal[i],T) for i in range(robot.n)]

traj = ClippedTrajectory(traj_fun, T)
loop = FBL(robot, env, [0,-9.81,0])

loop.setR(reference = traj, goal = goal, threshold = 0.1)
loop.setK(kp = [16,8], kd = [4,2])

q_d, qd_d, qdd_d = loop.reference(2, 1)

In [ ]:
import numpy as np
ref = np.array([1,1,2])#.reshape(3,1)
pos = np.array([1.04,1.04,2.04])#.reshape(3,1)
th = 0.05

pos.dot(ref)

e = ref.T - pos.T
e

go = (e < th)[False].shape[0]
go

In [ ]:
from tools.Models import *
from math import pi
robot = OneLink()
q = pi/2
print(robot.inertia(q))
robot.gravload(q, gravity = [0,-9.81, 0])

In [92]:
from tools.Models import *
from math import pi
import sympy as sym

robot = SymbolicTwoLink()
q = sym.symbols("q1:3")
A = robot[1].A(q[1])
print(A.inv().R)
robot.links[1].isrevolute

[[cos(q2) sin(q2) 0]
 [-sin(q2) cos(q2) 0]
 [0 0 1]]


True

In [3]:
import sympy as sym
import numpy as np

n = 3
Ixx = sym.symbols(f"Ixx(1:{n+1})")
Iyy = sym.symbols(f"Iyy(1:{n+1})")
Izz = sym.symbols(f"Izz(1:{n+1})")

print(Izz)


I = np.zeros((3,3,3),dtype=object)
I[1,:,:] = np.diag([Ixx[0],Iyy[0],Izz[0]])
I[1,:,:]

(Izz1, Izz2, Izz3)


array([[Ixx1, 0, 0],
       [0, Iyy1, 0],
       [0, 0, Izz1]], dtype=object)

In [7]:
import sympy as sym
import numpy as np
from tools.Models import *

robot = SymbolicTwoLink()
q = sym.symbol("q1:3")
A = robot[1].A(q[1])

n = 2
a = sym.symbol(f"a(1:{n+1})") #link lenghts
dc = sym.symbol(f"dc(1:{n+1})")

ri = np.zeros((3,n+1),dtype=object)
print(ri)
ri[:,2] = A.t
print(ri[:,2])
print(a[1])
print(ri)
ri[:,2] = [ok.subs(a[1],dc[1]) for ok in ri[:,2]]
print(ri)

[[0 0 0]
 [0 0 0]
 [0 0 0]]
[a2*cos(q2) a2*sin(q2) 0]
a2
[[0 0 a2*cos(q2)]
 [0 0 a2*sin(q2)]
 [0 0 0]]
[[0 0 dc2*cos(q2)]
 [0 0 dc2*sin(q2)]
 [0 0 0]]


In [49]:
import sympy as sym
import numpy as np
from tools.Models import *

robot = SymbolicTwoLink()
n = 2
q = sym.symbol(f"q(1:{n+1})") #link variables
a = sym.symbol(f"a(1:{n+1})") #link lenghts

#dynamic parameters
m = sym.symbol(f"a(1:{n+1})") #link masses
dc = sym.symbol(f"dc(1:{n+1})")
Ixx = sym.symbol(f"Ixx(1:{n+1})")
Iyy = sym.symbol(f"Iyy(1:{n+1})")
Izz = sym.symbol(f"Izz(1:{n+1})") 

I = np.full((n,3,3), sym.zero(), dtype = object) #tensor with ith matrix representing inertia matrix of ith COM
ri = np.full((n, 3), sym.zero(), dtype = object) #n vectors from RF i-1 to i wrt RF i-1
rc = np.full((n, 3), sym.zero(), dtype = object) #n position vectors of COM i seen from RF i
rim1c = np.full((n, 3), sym.zero(), dtype = object) #n vectors from RF i-1 to COM as seen from RF i
riim1 = np.full((n, 3), sym.zero(), dtype = object) #n vectors from RF i to RF i-1 as seen from RF i
Rinv = np.full((n,3,3), sym.zero(), dtype = object) #tensor with ith matrix representing rotation from Rf i to Rf i-1
   
print(riim1)   
# for i in range(n):
i = 0
sigma = robot.links[i].isprismatic #check for prismatic joints
I[i,:,:] = np.diag([Ixx[i],Iyy[i],Izz[i]]) #diagonal inertias
A = robot[i].A(q[i])
ri[i,:] = A.t #homogeneus from frame i to i+1
Ainv = A.inv()
Rinv[i,:,:] = Ainv.R
riim1[i,:] = Ainv.t

ri[ri == 0] = sym.zero()
rim1c = Rinv[i,:,:] @ [elem.subs(a[i],dc[i]) for elem in ri[i,:]]

print(riim1.shape)
print([elem.subs(q[i],dc[i]) for elem in riim1[i,:]])

[[0 0 0]
 [0 0 0]]
(2, 3)
[-a1*sin(dc1)**2 - a1*cos(dc1)**2, 0, 0]


In [18]:
import sympy as sym
import numpy as np
from tools.Models import *

robot = TwoLink()
n = 2
q = sym.symbol(f"q(1:{n+1})") #link variables
a = sym.symbol(f"a(1:{n+1})") #link lenghts

#dynamic parameters
m = sym.symbol(f"a(1:{n+1})") #link masses
dc = sym.symbol(f"dc(1:{n+1})")
Ixx = sym.symbol(f"Ixx(1:{n+1})")
Iyy = sym.symbol(f"Iyy(1:{n+1})")
Izz = sym.symbol(f"Izz(1:{n+1})") 

I = np.zeros((n,3,3),dtype=object) #tensor with ith matrix representing inertia matrix of ith COM
ri = np.full((n, 3), sym.zero(), dtype = object) #n vectors from RF i-1 to i wrt RF i-1
print(type(ri[0,2]))
rc = np.zeros((n,3),dtype=object) #n position vectors of COM i seen from RF i
rim1c = np.zeros((n,3),dtype=object) #n vectors from RF i-1 to COM as seen from RF i
riim1 = np.zeros((n,3),dtype=object) #n vectors from RF i to RF i-1 as seen from RF i
Rinv = np.zeros((n,3,3),dtype=object) #tensor with ith matrix representing rotation from Rf i to Rf i-1
   
# for i in range(n):
i = 0
sigma = robot.links[i].isprismatic #check for prismatic joints
I[i,:,:] = np.diag([Ixx[i],Iyy[i],Izz[i]]) #diagonal inertias
A = robot[i].A(q[i]); 
ri[i,:] = A.t #homogeneus from frame i to i+1
Ainv = A.inv(); Rinv[i,:,:] = Ainv.R; riim1 = Ainv.t

print(type(ri[0,2]))
print([elem for elem in ri[i,:]])
rim1c = Rinv[i,:,:] @ [elem.subs(a[i],dc[i]) for elem in ri[i,:]]
# rc[i,:] = riim1 + rim1c
    

<class 'sympy.core.numbers.Zero'>


ValueError: can't insert different type of object